# Homework 3

**Name:** -- Víctor Manuel Mariscal Cervantes --

**e-mail:** -- victor.mariscal4459@alumnos.udg.mx --

# Activity 1: Path Length Comparison (BM1 vs BM2 vs CRW)

This activity compares the total path length of different motion models:
- **BM1:** Brownian Motion with one configuration.
- **BM2:** Brownian Motion with a different configuration.
- **CRW:** Correlated Random Walk with specific parameters.

### Steps
1. **Trajectory Generation:** Generate trajectories for BM1, BM2, and CRW.
2. **Path Length Calculation:** For each trajectory, compute the total path length.
3. **Visualization:** Plot a linear progression (cumulative path length over steps) using Plotly.

The cumulative path length is computed by summing the Euclidean distances between consecutive positions:

$$ L = \sum_{i=1}^{N-1} \sqrt{(x_{i+1}-x_i)^2 + (y_{i+1}-y_i)^2} $$

Below is the complete code for Activity 1.

In [ ]:
# Import necessary libraries
import math
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import random
import os

In [ ]:
# ----------------------------
# 2D Vector Class Definition
# ----------------------------
class Vec2d(object):
    __slots__ = ['x', 'y']
    
    def __init__(self, x_or_pair, y=None):
        if y is None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    def rotated(self, angle):
        cos_a = math.cos(angle)
        sin_a = math.sin(angle)
        return Vec2d(self.x * cos_a - self.y * sin_a, self.x * sin_a + self.y * cos_a)

# ----------------------------
# Trajectory Generation Functions
# ----------------------------
def bm_2d(n_steps=1000, speed=5, s_pos=[0, 0]):
    """
    Generate a 2D Brownian Motion trajectory.
    
    Parameters:
        n_steps (int): Number of steps.
        speed (float): Step size.
        s_pos (list): Starting position [x, y].
    
    Returns:
        pd.DataFrame: Trajectory with columns 'x_pos' and 'y_pos'.
    """
    positions = [s_pos]
    velocity = Vec2d(speed, 0)
    for _ in range(n_steps - 1):
        turn_angle = np.random.uniform(-np.pi, np.pi)
        velocity = velocity.rotated(turn_angle)
        new_pos = [positions[-1][0] + velocity.x, positions[-1][1] + velocity.y]
        positions.append(new_pos)
    return pd.DataFrame(positions, columns=['x_pos', 'y_pos'])

def crw_2d(n_steps=1000, speed=5, s_pos=[0, 0], sigma=np.pi/8):
    """
    Generate a 2D Correlated Random Walk trajectory.
    
    Parameters:
        n_steps (int): Number of steps.
        speed (float): Step size.
        s_pos (list): Starting position [x, y].
        sigma (float): Standard deviation for turning angle noise.
    
    Returns:
        pd.DataFrame: Trajectory with columns 'x_pos' and 'y_pos'.
    """
    positions = [s_pos]
    current_angle = 0  # initial direction
    for _ in range(n_steps - 1):
        turn_angle = np.random.normal(0, sigma)
        current_angle += turn_angle
        new_pos = [positions[-1][0] + speed * math.cos(current_angle),
                   positions[-1][1] + speed * math.sin(current_angle)]
        positions.append(new_pos)
    return pd.DataFrame(positions, columns=['x_pos', 'y_pos'])

# ----------------------------
# Path Length Calculation Function
# ----------------------------
def compute_path_length(df):
    """
    Calculate the total path length from a trajectory DataFrame.
    
    Parameters:
        df (pd.DataFrame): DataFrame with 'x_pos' and 'y_pos'.
    
    Returns:
        float: Total path length.
    """
    diffs = df.diff().dropna()  # differences between consecutive positions
    segment_lengths = np.sqrt(diffs['x_pos']**2 + diffs['y_pos']**2)
    return segment_lengths.sum()

# ----------------------------
# Automatic Generation of Trajectory Configurations
# ----------------------------
# Define the number of BM and CRW trajectories to generate.
num_bm = 5   # Number of Brownian Motion trajectories
num_crw = 3  # Number of Correlated Random Walk trajectories

trajectory_configs = []

# Generate BM configurations with random speeds between 1 and 20, including the speed in the label.
bm_speeds = random.sample(range(1, 21), num_bm)  # Unique speeds for BM trajectories
for i, speed in enumerate(bm_speeds):
    trajectory_configs.append({'label': f'BM{i+1} - {speed}', 'type': 'BM', 'speed': speed})

# Generate CRW configurations with random speeds between 1 and 20, ensuring unique speeds.
crw_speeds = random.sample(range(1, 21), num_crw)  # Unique speeds for CRW trajectories
for i, speed in enumerate(crw_speeds):
    trajectory_configs.append({'label': f'CRW{i+1} - {speed}', 'type': 'CRW', 'speed': speed, 'sigma': np.pi/8})

# Common parameters for all trajectories
n_steps = 1000
start_pos = [0, 0]

# ----------------------------
# Generate Trajectories and Compute Their Path Lengths
# ----------------------------
trajectories = {}
total_lengths = {}

for config in trajectory_configs:
    label = config['label']
    if config['type'] == 'BM':
        df = bm_2d(n_steps=n_steps, speed=config['speed'], s_pos=start_pos)
    elif config['type'] == 'CRW':
        df = crw_2d(n_steps=n_steps, speed=config['speed'], s_pos=start_pos, sigma=config['sigma'])
    trajectories[label] = df
    total_lengths[label] = compute_path_length(df)

# ----------------------------
# Plotting: Cumulative Path Length vs. Step Number
# ----------------------------
# For each trajectory, create a linear progression (linearly interpolated) from 0 to its final path length.
steps = np.arange(n_steps)
fig = go.Figure()

for label, length in total_lengths.items():
    cumulative_line = np.linspace(0, length, n_steps)
    fig.add_trace(go.Scatter(x=steps, y=cumulative_line, mode='lines', name=label))
    # Annotate the final value for each trajectory
    fig.add_annotation(x=n_steps-1, y=length, text=f"{length:.2f}", showarrow=True, arrowhead=1)

fig.update_layout(title='Cumulative Path Length over Steps',
                  xaxis_title='Step Number',
                  yaxis_title='Cumulative Path Length')
fig.show()

# Save the figure image
output_dir = 'notebook_images'
os.makedirs(output_dir, exist_ok=True)
fig.write_image(os.path.join(output_dir, 'activity_1_path_length.png'))

# Activity 2: Mean Squared Displacement (MSD) Comparison for BM vs CRW

In this activity we compute the Mean Squared Displacement (MSD) for each trajectory.
For each trajectory, the MSD is calculated as:

$$ MSD(t) = (x(t)-x(0))^2 + (y(t)-y(0))^2 $$

The code generates multiple BM and CRW trajectories (as configured), computes their individual MSD curves, 
and plots them for comparison using Plotly.

In [ ]:
# ---------------------------------------------------------------
# Number of trajectories to compare (adjust these values as desired)
NUM_BM_TRAJ = 2    # <-- Change this value to adjust the number of BM trajectories to compare
NUM_CRW_TRAJ = 2   # <-- Change this value to adjust the number of CRW trajectories to compare
# ---------------------------------------------------------------

# Function to compute the Mean Squared Displacement (MSD) for a given trajectory.
# The MSD at each step is computed as the squared distance from the starting position.
def compute_msd(df):
    x0 = df['x_pos'].iloc[0]
    y0 = df['y_pos'].iloc[0]
    msd = (df['x_pos'] - x0)**2 + (df['y_pos'] - y0)**2
    return msd.values

# Common parameters
n_steps = 1000
start_pos = [0, 0]

# Lists to store individual MSD curves for BM and CRW
msd_bm_curves = []
msd_crw_curves = []

# Generate BM trajectories and compute their MSD curves individually
for i in range(NUM_BM_TRAJ):
    df_bm = bm_2d(n_steps=n_steps, speed=5, s_pos=start_pos)
    msd_bm = compute_msd(df_bm)
    msd_bm_curves.append(msd_bm)

# Generate CRW trajectories and compute their MSD curves individually
for i in range(NUM_CRW_TRAJ):
    df_crw = crw_2d(n_steps=n_steps, speed=5, s_pos=start_pos, sigma=np.pi/8)
    msd_crw = compute_msd(df_crw)
    msd_crw_curves.append(msd_crw)

# Create time vector (number of steps)
time = np.arange(n_steps)

# Plotting each individual MSD curve for BM and CRW using Plotly
fig = go.Figure()

# Add BM MSD curves
for idx, msd in enumerate(msd_bm_curves, start=1):
    fig.add_trace(go.Scatter(x=time, y=msd, mode='lines', name=f'BM {idx}'))

# Add CRW MSD curves
for idx, msd in enumerate(msd_crw_curves, start=1):
    fig.add_trace(go.Scatter(x=time, y=msd, mode='lines', name=f'CRW {idx}'))

fig.update_layout(title='MSD Comparison for BM vs CRW',
                  xaxis_title='Time (steps)',
                  yaxis_title='Mean Squared Displacement (MSD)')
fig.show()

# Save the MSD figure
os.makedirs('notebook_images', exist_ok=True)
fig.write_image(os.path.join('notebook_images', 'activity_2_msd.png'))

# Activity 3: Turning-Angle Distribution (Source vs. Observed) for CRW

In this activity we analyze the distribution of turning angles in a Correlated Random Walk (CRW).
The observed turning angles (in degrees) are computed from the trajectory and compared with the 
expected (source) distribution given by the Cauchy probability density function:

$$ f(\theta) = \frac{1}{\pi \gamma \left(1 + \left(\frac{\theta}{\gamma}\right)^2\right)} $$

The expected curves are scaled so that their peaks are visible relative to the histogram of observed angles.

In [ ]:
# ----------------------------
# New Function: CRW using Cauchy-distributed turning angles
# ----------------------------
def crw_cauchy_2d(n_steps=1000, speed=5, s_pos=[0, 0], cauchy_coeff=0.3):
    """
    Generate a 2D Correlated Random Walk (CRW) trajectory where turning angles are 
    drawn from a Cauchy distribution with a given scale (cauchy_coeff in radians).
    
    Parameters:
        n_steps (int): Number of steps.
        speed (float): Step size.
        s_pos (list): Starting position [x, y].
        cauchy_coeff (float): Scale parameter for the Cauchy distribution (in radians).
    
    Returns:
        pd.DataFrame: Trajectory with columns 'x_pos' and 'y_pos'.
    """
    positions = [s_pos]
    current_angle = 0  # initial direction
    for _ in range(n_steps - 1):
        # Draw turning angle from a Cauchy distribution
        turning_angle = cauchy_coeff * np.random.standard_cauchy()
        current_angle += turning_angle
        new_pos = [positions[-1][0] + speed * math.cos(current_angle),
                   positions[-1][1] + speed * math.sin(current_angle)]
        positions.append(new_pos)
    return pd.DataFrame(positions, columns=['x_pos', 'y_pos'])

# ----------------------------
# Function: Compute turning angles for a trajectory
# ----------------------------
def compute_turning_angles(df):
    """
    Compute turning angles (in degrees) between consecutive displacement vectors in a trajectory.
    
    Parameters:
        df (pd.DataFrame): Trajectory with columns 'x_pos' and 'y_pos'.
    
    Returns:
        np.array: Array of turning angles in degrees.
    """
    # Compute displacement vectors
    dx = df['x_pos'].diff().dropna().values
    dy = df['y_pos'].diff().dropna().values
    # Compute angles of each displacement vector (in radians)
    angles = np.arctan2(dy, dx)
    # Compute differences between consecutive angles
    turning_angles = np.diff(angles)
    # Normalize to the range [-pi, pi]
    turning_angles = (turning_angles + np.pi) % (2 * np.pi) - np.pi
    # Convert to degrees
    turning_angles_deg = np.degrees(turning_angles)
    return turning_angles_deg

# ----------------------------
# Function: Cauchy PDF for the source distribution (in degrees)
# ----------------------------
def cauchy_pdf(theta, gamma_deg):
    """
    Compute the Cauchy probability density function for angles (in degrees).
    
    Parameters:
        theta (array-like): Angles in degrees.
        gamma_deg (float): Scale parameter in degrees.
    
    Returns:
        np.array: PDF values.
    """
    # Standard Cauchy PDF: $$ f(\theta)= \frac{1}{\pi \gamma \left(1 + \left(\frac{\theta}{\gamma}\right)^2\right)} $$
    return 1 / (np.pi * gamma_deg * (1 + (theta / gamma_deg)**2))

# ----------------------------
# Parameters and CRW Trajectory Generation
# ----------------------------
n_steps = 1000
start_pos = [0, 0]

# Define two different Cauchy coefficients (scale parameters in radians) for CRW trajectories
cauchy_coeff1 = 0.3  # First CRW trajectory
cauchy_coeff2 = 0.8  # Second CRW trajectory

# Generate two CRW trajectories using the Cauchy-distributed turning angles
crw_traj1 = crw_cauchy_2d(n_steps=n_steps, speed=5, s_pos=start_pos, cauchy_coeff=cauchy_coeff1)
crw_traj2 = crw_cauchy_2d(n_steps=n_steps, speed=5, s_pos=start_pos, cauchy_coeff=cauchy_coeff2)

# Compute observed turning angles (in degrees) for each trajectory
turning_angles1 = compute_turning_angles(crw_traj1)
turning_angles2 = compute_turning_angles(crw_traj2)

# Convert the Cauchy coefficients from radians to degrees for the expected (source) distribution
gamma_deg1 = np.degrees(cauchy_coeff1)
gamma_deg2 = np.degrees(cauchy_coeff2)

# ----------------------------
# Compute Theoretical PDFs over full range (-180 to 180)
# ----------------------------
x_min, x_max = -180, 180
theta_range = np.linspace(x_min, x_max, 361)
pdf1 = cauchy_pdf(theta_range, gamma_deg1)
pdf2 = cauchy_pdf(theta_range, gamma_deg2)

# ----------------------------
# Scaling: Manually scale the theoretical PDFs so that their peaks are visible with the histogram.
# The histograms show peak density values ~25, while the theoretical PDFs are around 0.025.
# We use a fixed scaling factor (e.g., 1000) to bring the curves into the same range.
# ----------------------------
fixed_scale = 1000
pdf1_scaled = pdf1 * fixed_scale
pdf2_scaled = pdf2 * fixed_scale

# ----------------------------
# Plotting: Observed vs. Expected Turning-Angle Distribution
# ----------------------------
fig = go.Figure()

# Plot histogram for observed turning angles (CRW trajectory 1)
fig.add_trace(go.Histogram(
    x=turning_angles1,
    nbinsx=100,
    xbins=dict(start=x_min, end=x_max, size=3),
    histnorm='density',
    name=f'Observed CRW1 - {cauchy_coeff1} (γ={gamma_deg1:.2f}°)',
    opacity=0.6
))
# Plot histogram for observed turning angles (CRW trajectory 2)
fig.add_trace(go.Histogram(
    x=turning_angles2,
    nbinsx=100,
    xbins=dict(start=x_min, end=x_max, size=3),
    histnorm='density',
    name=f'Observed CRW2 - {cauchy_coeff2} (γ={gamma_deg2:.2f}°)',
    opacity=0.6
))

fig.add_trace(go.Scatter(
    x=theta_range,
    y=pdf1_scaled,
    mode='lines',
    name=f'Expected CRW1 - {cauchy_coeff1} (γ={gamma_deg1:.2f}°)',
    line=dict(color='blue', width=3),
    fill='tozeroy',
    fillcolor='rgba(0, 0, 255, 0.3)'
))
fig.add_trace(go.Scatter(
    x=theta_range,
    y=pdf2_scaled,
    mode='lines',
    name=f'Expected CRW2 - {cauchy_coeff2} (γ={gamma_deg2:.2f}°)',
    line=dict(color='red', width=3),
    fill='tozeroy',
    fillcolor='rgba(255, 0, 0, 0.3)'
))

fig.update_layout(
    title='Observed vs. Expected Turning-Angle Distribution for CRW',
    xaxis_title='Turning Angle (degrees)',
    yaxis_title='Probability Density',
    barmode='overlay',
    xaxis=dict(range=[x_min, x_max])
)
fig.update_traces(opacity=0.75)
fig.show()

# Save the Turning-Angle figure
os.makedirs('notebook_images', exist_ok=True)
fig.write_image(os.path.join('notebook_images', 'activity_3_turning_angles.png'))

# Activity 4: Step-Length Distribution (Source vs. Observed) for Lévy Walk

In this activity, we compare the distribution of step lengths from a Lévy Walk with the expected source distribution.

### Steps
1. **Trajectory Generation:** Two Lévy Walk trajectories are generated usando una transformación de la distribución Pareto, 
   con parámetros **α (alpha)** y **β (beta)** (ambos en el rango [0.1, 1]).
2. **Step-Length Calculation:** Se calculan las distancias euclidianas entre posiciones consecutivas.
3. **Theoretical PDF:** Se utiliza la siguiente fórmula para la PDF teórica:

$$ f_Y(y) = \frac{\alpha}{\beta} \frac{x_m^{\alpha/\beta}}{y^{\alpha/\beta+1}} \quad \text{para } y \ge x_m $$

   donde $x_m$ es el paso mínimo (step_scale).
4. **Scaling:** Se aplica un factor de escala fijo (e.g., 1000) para que la curva teórica sea visible en comparación con el histograma.
5. **Ejes:** Se determinan los límites del eje x en función de la moda de la distribución observada, mostrando desde 2 unidades antes hasta 5 unidades después.
6. **Visualización:** Se plotean los histogramas de las distribuciones observadas (con intervalos de 0.1) y se superponen las curvas teóricas suavizadas.

Las leyendas de los histogramas muestran ambos parámetros (**α** y **β**), mientras que las leyendas de las curvas esperadas muestran solo **α**.

In [ ]:
# ----------------------------
# Function: Generate a Lévy Walk trajectory (LW) with parameters alpha and beta.
# Both alpha and beta must be in the range [0.1, 1].
# The step length is generated as: 
#    step_length = step_scale * (1 + Pareto(alpha))^beta
# ----------------------------
def levy_walk_2d(n_steps=1000, alpha=0.5, beta=0.3, s_pos=[0, 0], step_scale=1):
    """
    Generate a 2D Lévy Walk trajectory.
    
    Parameters:
        n_steps (int): Number of steps.
        alpha (float): Power-law exponent for the Pareto part (in [0.1,1]).
        beta (float): Exponent to transform the Pareto value (in [0.1,1]).
        s_pos (list): Starting position [x, y]. <-- Modify this to change the starting point.
        step_scale (float): Scale factor for step lengths (minimum step length).
        
    Returns:
        pd.DataFrame: Trajectory with columns 'x_pos' and 'y_pos'.
    """
    positions = [s_pos]
    for _ in range(n_steps - 1):
        # Sample from Pareto: np.random.pareto(alpha) returns values > 0;
        # adding 1 makes the minimum value 1, then raising to beta and scaling.
        step_length = step_scale * (1 + np.random.pareto(alpha))**beta
        turning_angle = np.random.uniform(-np.pi, np.pi)
        new_x = positions[-1][0] + step_length * math.cos(turning_angle)
        new_y = positions[-1][1] + step_length * math.sin(turning_angle)
        positions.append([new_x, new_y])
    return pd.DataFrame(positions, columns=['x_pos', 'y_pos'])

# ----------------------------
# Function: Compute step lengths for a given trajectory.
# ----------------------------
def compute_step_lengths(df):
    """
    Compute the step lengths (Euclidean distances) between consecutive positions.
    
    Parameters:
        df (pd.DataFrame): Trajectory with columns 'x_pos' and 'y_pos'.
        
    Returns:
        np.array: Array of step lengths.
    """
    diffs = df.diff().dropna()
    step_lengths = np.sqrt(diffs['x_pos']**2 + diffs['y_pos']**2)
    return step_lengths.values

# ----------------------------
# Function: Theoretical PDF for the transformed Pareto (Lévy) distribution.
# If X ~ Pareto(alpha) with xm = 1 (i.e., X = 1 + Pareto(alpha)), and 
# Y = step_scale * X^beta, then the PDF of Y is:
#    $$ f_Y(y)= \frac{\alpha}{\beta} \frac{\;x_m^{\alpha/\beta}}{y^{\alpha/\beta+1}} \quad \text{for } y \ge x_m $$
# ----------------------------
def levy_pdf(l, alpha, beta, xm=1):
    """
    Compute the theoretical PDF for Lévy walk step lengths.
    
    Parameters:
        l (array-like): Step lengths.
        alpha (float): Parameter alpha.
        beta (float): Parameter beta.
        xm (float): Minimum step length (scale), default 1.
    
    Returns:
        np.array: PDF values for l >= xm (0 for l < xm).
    """
    l = np.atleast_1d(l)
    pdf = np.zeros_like(l, dtype=float)
    mask = l >= xm
    pdf[mask] = (alpha/beta) * (xm**(alpha/beta)) / (l[mask]**(alpha/beta + 1))
    return pdf

# ----------------------------
# Parameters and LW Trajectory Generation
# ----------------------------
n_steps = 1000
start_pos = [0, 0]       # <-- Change this to set a different starting point.
step_scale = 1           # Minimum step length.

# Define two LW trajectories with different parameters (alpha and beta in [0.1, 1]).
alpha1, beta1 = 0.5, 0.3  # First LW trajectory parameters.
alpha2, beta2 = 0.8, 0.6  # Second LW trajectory parameters.

# Generate trajectories.
lw_traj1 = levy_walk_2d(n_steps=n_steps, alpha=alpha1, beta=beta1, s_pos=start_pos, step_scale=step_scale)
lw_traj2 = levy_walk_2d(n_steps=n_steps, alpha=alpha2, beta=beta2, s_pos=start_pos, step_scale=step_scale)

# Compute observed step lengths.
step_lengths1 = compute_step_lengths(lw_traj1)
step_lengths2 = compute_step_lengths(lw_traj2)

# ----------------------------
# Compute Theoretical PDFs for step lengths.
# Use the full range from 0 to the 99th percentile of the combined observed step lengths.
# ----------------------------
combined_steps = np.concatenate((step_lengths1, step_lengths2))
l_min = 0  # Start at 0.
l_max = np.percentile(combined_steps, 99)
l_range = np.linspace(l_min, l_max, 1000)  # More points for a smoother curve.

pdf_lw1 = levy_pdf(l_range, alpha1, beta1, xm=step_scale)
pdf_lw2 = levy_pdf(l_range, alpha2, beta2, xm=step_scale)

# ----------------------------
# Scaling: Apply a fixed scaling factor so that theoretical curves are visible relative to histograms.
# ----------------------------
fixed_scale = 1000  # <-- Adjust this scaling factor as needed.
pdf_lw1_scaled = pdf_lw1 * fixed_scale
pdf_lw2_scaled = pdf_lw2 * fixed_scale

# ----------------------------
# Determine x-axis limits based on "important values":
# Compute the mode (bin with highest density) from the combined observed step lengths,
# then set x-axis limits as [mode - 2, mode + 5].
# ----------------------------
bins_mode = np.arange(step_scale, l_max + 0.1, 0.1)
hist_comb, bin_edges = np.histogram(combined_steps, bins=bins_mode, density=True)
max_bin_idx = np.argmax(hist_comb)
mode_x = (bin_edges[max_bin_idx] + bin_edges[max_bin_idx + 1]) / 2
x_lower = mode_x - 2
x_upper = mode_x + 5

# Ensure lower limit is not negative.
if x_lower < 0:
    x_lower = 0

# ----------------------------
# Plotting: Observed vs. Expected Step-Length Distribution for Lévy Walk.
# Observed legends include both alpha and beta; expected legends show only alpha.
# Histogram bins are defined with a step of 0.1 and thin borders.
# ----------------------------
bins_hist = np.arange(step_scale, l_max + 0.1, 0.1)

fig = go.Figure()

# Observed LW trajectory 1.
fig.add_trace(go.Histogram(
    x=step_lengths1,
    xbins=dict(start=step_scale, end=l_max, size=0.1),
    histnorm='density',
    name=f'Observed LW1 (α={alpha1}, β={beta1})',
    opacity=0.6,
    marker_line=dict(width=1, color='black')
))
# Observed LW trajectory 2.
fig.add_trace(go.Histogram(
    x=step_lengths2,
    xbins=dict(start=step_scale, end=l_max, size=0.1),
    histnorm='density',
    name=f'Observed LW2 (α={alpha2}, β={beta2})',
    opacity=0.6,
    marker_line=dict(width=1, color='black')
))

# Smooth the expected curves using a moving average (window size = 5) for aesthetics.
window = 5
pdf_lw1_smooth = np.convolve(pdf_lw1_scaled, np.ones(window)/window, mode='same')
pdf_lw2_smooth = np.convolve(pdf_lw2_scaled, np.ones(window)/window, mode='same')

# Expected (source) curve for LW trajectory 1.
fig.add_trace(go.Scatter(
    x=l_range,
    y=pdf_lw1_smooth,
    mode='lines',
    name=f'Expected LW1 (α={alpha1})',
    line=dict(color='blue', width=3),
    fill='tozeroy',
    fillcolor='rgba(0, 0, 255, 0.3)'
))
# Expected (source) curve for LW trajectory 2.
fig.add_trace(go.Scatter(
    x=l_range,
    y=pdf_lw2_smooth,
    mode='lines',
    name=f'Expected LW2 (α={alpha2})',
    line=dict(color='red', width=3),
    fill='tozeroy',
    fillcolor='rgba(255, 0, 0, 0.3)'
))

fig.update_layout(
    title='Observed vs. Expected Step-Length Distribution for Lévy Walk',
    xaxis_title='Step Length',
    yaxis_title='Probability Density',
    barmode='overlay',
    xaxis=dict(range=[x_lower, x_upper])
)
fig.update_traces(opacity=0.75)
fig.show()

# Save the Step-Length Distribution figure
os.makedirs('notebook_images', exist_ok=True)
fig.write_image(os.path.join('notebook_images', 'activity_4_step_length.png'))

# Conclusions

In this homework we implemented four activities to study different aspects of random motion:

- **Activity 1:** We generated trajectories for Brownian Motion and Correlated Random Walks, computed the cumulative path length,
  and visualized the progression using Plotly. The total path length is given by:
  $$ L = \sum_{i=1}^{N-1} \sqrt{(x_{i+1}-x_i)^2 + (y_{i+1}-y_i)^2} $$

- **Activity 2:** We computed the Mean Squared Displacement (MSD) for BM and CRW trajectories as:
  $$ MSD(t) = (x(t)-x(0))^2 + (y(t)-y(0))^2 $$
  and compared the evolution of the MSD over time.

- **Activity 3:** We analyzed the turning-angle distribution for CRW trajectories. The expected Cauchy distribution is given by:
  $$ f(\theta)= \frac{1}{\pi \gamma \left(1 + \left(\frac{\theta}{\gamma}\right)^2\right)} $$

- **Activity 4:** We generated Lévy Walk trajectories using a transformed Pareto distribution with parameters \( \alpha \) and \( \beta \).
  The theoretical PDF is:
  $$ f_Y(y)= \frac{\alpha}{\beta}\frac{x_m^{\alpha/\beta}}{y^{\alpha/\beta+1}} \quad \text{for } y \ge x_m $$

For each activity, we not only visualized the figures interactively with Plotly but also saved them as image files
in the directory `notebook_images` (which is created automatically if it does not exist), so that the figures
can be visualized externally.

This notebook demonstrates the use of stochastic processes in simulating random motion and analyzing
their statistical properties, with detailed visualizations and reproducible code.